<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P4_ChatGLM3(zhipuAI)_base%26chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
先加载chat模型查看数据格式（chatglm3-6b而不是chatglm3-base)模型
使用 ?model.chat 代码  会返回文件，点击文件进去源码  or  进入模型所在文件夹查看源码
chatglm3对special token进行了特殊处理，没法通过tokenize方式得到，只能使用get_command得到token（千问也对special token进行了特殊处理）避免decode混淆
chatglm3有一个后处理的功能（因为它有functioncall和执行代码的功能）

In [ ]:
basemodel推荐把embedding层也训练，这里只训练了注意力层

In [ ]:
GLM的chat格式：在base的基础上进行chat的训练，V1是prefix训练，V2是CausalLM训练，prefix训练在多轮对话上效率不高（因为要把QAQA都拆开来训练）
chatglm3支持function call（工具调用）和code interpreter（代码执行）


In [ ]:
#使用chat模型查看数据格式：
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("d:/Pretrained_models/ZhipuAI/chatglm3-6b/", trust_remote_code=True)
model = AutoModel.from_pretrained("d:/Pretrained_models/ZhipuAI/chatglm3-6b/", trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype=torch.half, device_map="auto")
model.chat(tokenizer, "考试的技巧有哪些？", history=[])

In [ ]:
?model.chat    #查看源码，通过源码确定数据需要被处理成为的格式

In [ ]:
?tokenizer.build_chat_input  #从源码看到了build_chat_input,这里就具体查看这个方法

In [ ]:
print(tokenizer("<|user|>", add_special_tokens=False))
print(tokenizer.get_command("<|user|>")) #确定special token的取用方法（tokenizer.get_command）

In [ ]:
tokenizer.build_chat_input("考试的技巧有哪些？", history=[], role="user")

In [ ]:
tokenizer.decode([64790, 64792, 64795, 30910,    13, 30910, 32227, 54530, 33741, 34953,
         31514, 64796])

数据格式：

[gMASK]sop<|user|> \n Prompt<|assistant|> \n Response eos_token
后面这个\n Response eos_token 是因为GLM3有后处理，会split('\n')来分割metadata(function名)和context(模型的回答) 在这里我们先不考虑functioncall和执行代码那部分的训练，因此用空的加一个\n来隔开response

In [ ]:
#确定格式后对base model进行训练：
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import torch
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

In [ ]:
ds = load_dataset("zhengr/alpaca-chinese-dataset")
tokenizer = AutoTokenizer.from_pretrained("d:/Pretrained_models/ZhipuAI/chatglm3-6b-base/", trust_remote_code=True)
print(tokenizer(tokenizer.eos_token), tokenizer.eos_token_id) #确定eostoken的取用方法（tokenizer.eos_token_id）

In [ ]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    query = "\n".join([example["instruction"], example["input"]]).strip()     # query
    query = tokenizer.build_chat_input(query, history=[], role="user")  # 格式变为[gMASK]sop<|user|> \n query<|assistant|> 并且是tensor形式
    response = tokenizer("\n" + example["output"], add_special_tokens=False)  # \n response, 缺少eos token；如果没有add_special_tokens=False就会在<|assistant|>后面又来一次[gMASK]sop
    input_ids = query["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id] #去掉query的tensor格式，加上eos tokend
    attention_mask = query["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1] #这个1是给eos token的
    labels = [-100] * len(query["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)

In [ ]:
#查看process_func是否正确
print(tokenizer.decode(tokenized_ds[1]["input_ids"]))
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"]))))

In [ ]:
model = AutoModelForCausalLM.from_pretrained("d:/Pretrained_models/ZhipuAI/chatglm3-6b-base/", trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype=torch.bfloat16, device_map="auto")
for name, param in model.named_parameters():
    print(name)
#查看模型都有哪些层

In [ ]:
#开始lora微调：
config = LoraConfig(target_modules=["query_key_value"], modules_to_save=["post_attention_layernorm"])  #basemodel推荐把embedding层也训练，这里只训练了注意力层
print(config)
model = get_peft_model(model, config)
for name, parameter in model.named_parameters():
    print(name)
print(model.print_trainable_parameters())
print(model)  #这里应该有lora层了，此时model是peftmodel
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=1e-4, #这里也可以调小一点
    remove_unused_columns=False,
    save_strategy="epoch"
)#因为在LoraConfig没有指定tasktype（task_type=TaskType.CAUSAL_LM），所以model作为peftmodel接受参数的形式是元组或者字典，必须加一句remove_unused_columns=False，否则训练的时候将无法正确接受参数
#源码中peftcausalmodel接受参数形式是指定了各参数的名称的，所以不需要加一句remove_unused_columns=False
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)
trainer.train()
model.eval()
print(model.chat(tokenizer, "考试的技巧有哪些？", history=[])[0])

In [ ]:
#量化8bit训练（显存占用减少，但训练速度会变慢）：  int8:-127 —— +127   需要安装bitsandbytes
#llama int8: 与llama之前的模型训练前面都一样，只是在model=要添加一个参数（load_in_8bit)：
model = AutoModelForCausalLM.from_pretrained("D:/Pretrained_models/modelscope/Llama-2-7b-ms", low_cpu_mem_usage=True,
                                             torch_dtype=torch.half, device_map="auto", load_in_8bit=True)
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype) #检验是否量化成功
model.config #会多一个quantization_config
#由于训练是int8，但是原模型是float32，所以不建议merge，而是使用PEFT调用

In [ ]:
#chatglm3 int8：同理
model = AutoModelForCausalLM.from_pretrained("d:/Pretrained_models/ZhipuAI/chatglm3-6b-base/", trust_remote_code=True, low_cpu_mem_usage=True,
                                             torch_dtype=torch.half, device_map="auto", load_in_8bit=True)
for name, param in model.named_parameters():
    print(name, param.dtype)